# 21.1类工厂函数

namedtuple就是一个类工厂函数的典范。
```{python}
Dog = namedtuple('Dog', 'name weight owner')  # Dog类，属性为name, weight, owner 
rex = Dog('Rex', 30, 'Bob')
```

我们来试着实现它。

In [23]:
def record_factory(cls_name, field_names):
    # 将field_names由字符串解析为tuple
    try:
        field_names = field_names.replace(',', ' ').split()
    except AttributeError:
        pass
    field_names = tuple(field_names)
    
    # 定义类的各个方法
    def __init__(self, *args, **kwargs):
        attrs = dict(zip(self.__slots__, args)) # 忽略一切slots中未包含的参数
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)
    
    def __iter__(self):
        for name in self.__slots__:
            yield getattr(self, name)
    
    def __repr__(self):
        values = ', '.join('{}={!r}'.format(*i) for i 
                           in zip(self.__slots__, self))
        return ('{}({})'.format(self.__class__.__name__, values))
    
    # 构造类
    cls_attrs = dict(__slots__=field_names,
                     __init__=__init__,
                     __iter__=__iter__,
                     __repr__=__repr__)
    return type(cls_name, (object,), cls_attrs)
        

Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 3.3, 'Mike')
rex

Dog(name='Rex', weight=3.3, owner='Mike')

关于type:
    type不仅可以作为函数使用，也可以作为类使用:
```        
MyClass = type('MyClass', (MySuperClass, MyMixin),
               {'x': 42, 'x2': lambda self: self.x * 2})
```
type的实例也是一个类。

# 21.3导入时和运行时比较

在导入时，解释器会从上到下一次性解析完 .py 模块的源码，然后生成用于执行的字节码。如果句法有错误，就在此时报告。
import语句不只是声明 ，在进程中首次导入模块时，还会运行所导入模块中的全部顶层代码——以后导入相同的模块则使用缓存，只做名称绑定。

在首次导入模块时，解释器会编译函数的定义体，把函数对象绑定到对应的全局名称上，但是显然解释器不会执行函数的定义体。
在导入时，解释器会执行每个类的定义体，甚至会执行嵌套类的定义体。执行类定义体的结果是，定义了类的属性和方法，并构建了类对象。



**关于本节的练习**：  
先计算被装饰的类 ClassThree 的定义体，然后运行装饰器函数。  
程序结束时，绑定在全局变量 one 上的 ClassOne 实例会被垃圾回收程序回收。  
类装饰器可能对子类没有影响。

# 21.4元类基础知识

object <--继承-- MyClass  
type <--实例-- MyClass


type是object的子类，object是type的实例  
type是自身的实例

In [42]:
print("object is an instance of: ",object.__class__)
print("type is the subclass of: ",type.__bases__)

object is an instance of:  <class 'type'>
type is the subclass of:  (<class 'object'>,)


所有元类都是type的子类，它们从type继承了实例化类的能力。  
**关于本节的练习**：    
创建 ClassFive 时调用了 MetaAleph.__init__ 方法。  
创建 ClassFive 的子类 ClassSix 时也调用了 MetaAleph.__init__ 方法。  
元类对其实例的子类会产生影响。